In [1]:
import ROOT

Welcome to JupyROOT 6.24/06


In [2]:
oF = ROOT.TFile.Open("/cmpp/Lecture2_Fitting/workspaces/NumberCounting/ws_combined_NumberCounting_model.root")
oF.ls()


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt

TFile**		/cmpp/Lecture2_Fitting/workspaces/NumberCounting/ws_combined_NumberCounting_model.root	
 TFile*		/cmpp/Lecture2_Fitting/workspaces/NumberCounting/ws_combined_NumberCounting_model.root	
  KEY: RooWorkspace	combined;1	combined
  KEY: TProcessID	ProcessID0;1	515c3ca0-98df-11ec-ae70-020011acbeef
  KEY: TDirectoryFile	SR_hists;1	SR_hists
  KEY: RooStats::HistFactory::Measurement	NumberCounting;1	NumberCounting


In [3]:
# Reading in the workspace
ws = oF.Get("combined")

# Reading in the elements needed for the fit
mu = ws.var("mu")
pdf = ws.pdf("simPdf")
sbModel = ws.obj("ModelConfig")
data = ws.data("obsData")
asimov = ws.data("asimovData")

# Configuring Minuit
ROOT.Math.MinimizerOptions.SetDefaultMinimizer("Minuit2")
ROOT.Math.MinimizerOptions.SetDefaultStrategy(0)
ROOT.Math.MinimizerOptions.SetDefaultPrintLevel(1)

# Create a single set containing NP and POI to feed negative log-likelihood (NLL)
params = ROOT.RooArgSet( sbModel.GetNuisanceParameters(), sbModel.GetParametersOfInterest() )

# Build negative log-likelihood (NLL)
theData = data
nll = pdf.createNLL( theData, ROOT.RooFit.Constrain(params), ROOT.RooFit.GlobalObservables(sbModel.GetGlobalObservables()), ROOT.RooFit.Offset(1) )

[#1] INFO:Minization -- p.d.f. provides expected number of events, including extended term in likelihood.
[#1] INFO:Minization -- createNLL: caching constraint set under name CONSTR_OF_PDF_simPdf_FOR_OBS_channelCat:obs_x_SR with 1 entries
[#1] INFO:Minization --  Including the following constraint terms in minimization: (lumiConstraint)
[#1] INFO:Minization -- The following global observables have been defined: (nominalLumi)
RooAbsTestStatistic::initSimMode: creating slave calculator #0 for state SR (1 dataset entries)
[#1] INFO:NumericIntegration -- RooRealIntegral::init(SR_model_Int[obs_x_SR]) using numeric integrator RooBinIntegrator to calculate Int(obs_x_SR)
[#1] INFO:Fitting -- RooAbsTestStatistic::initSimMode: created 1 slave calculators.


In [4]:
# Helper function to perform the fit
def minimize(fcn, save = False, retry_mode = 3):
  printLevel = ROOT.Math.MinimizerOptions.DefaultPrintLevel()
  msgLevel = ROOT.RooMsgService.instance().globalKillBelow()
  if printLevel < 0:
      ROOT.RooMsgService.instance().globalKillBelow(ROOT.RooFit.FATAL)
  
  strategy = ROOT.Math.MinimizerOptions.DefaultStrategy()
  save_def_strategy = strategy

  minimizer = ROOT.RooMinimizer(fcn)
  minimizer.optimizeConst(2)
  minimizer.setStrategy(strategy)
  minimizer.setPrintLevel(printLevel)
  minimizer.setMinimizerType(ROOT.Math.MinimizerOptions.DefaultMinimizerType())

  status = minimizer.minimize( ROOT.Math.MinimizerOptions.DefaultMinimizerType(), ROOT.Math.MinimizerOptions.DefaultMinimizerAlgo() )

  # Possibly re-trying if the fit didn't work
  if retry_mode == 0:
    if status != 0 and status != 1 and strategy < 2:
      strategy += 1
      logger.warning( f"Fit failed with status {status}. Retrying with strategy {strategy}" )
      minimizer.setStrategy(strategy)
      status = minimizer.minimize( ROOT.Math.MinimizerOptions.DefaultMinimizerType(), ROOT.Math.MinimizerOptions.DefaultMinimizerAlgo() )

    if status != 0 and status != 1 and strategy < 2:
      strategy += 1
      logger.warning( f"Fit failed with status {status}. Retrying with strategy {strategy}" )
      minimizer.setStrategy(strategy)
      status = minimizer.minimize( ROOT.Math.MinimizerOptions.DefaultMinimizerType(), ROOT.Math.MinimizerOptions.DefaultMinimizerAlgo() )

  else:
    for i in range(retry_mode):
      if status == 0 or status == 1: break
      logger.warning( f"Fit failed with status {status}. Retrying with strategy {strategy}." )
      minimizer.setStrategy(strategy)
      status = minimizer.minimize( ROOT.Math.MinimizerOptions.DefaultMinimizerType(), ROOT.Math.MinimizerOptions.DefaultMinimizerAlgo() )

  if printLevel < 0:
    ROOT.RooMsgService.insurance().setGlobalKillerBelow(msgLevel)
  ROOT.Math.MinimizerOptions.SetDefaultStrategy(save_def_strategy)

  if save:
    fitRes = minimizer.save( f"fitresult_{fcn.GetName()}", f"fitresult_{fcn.GetName()}" )
    return fitRes


In [5]:
res = minimize(nll, True, 0)
res.SaveAs("result.root")

[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_simPdf_obsData_with_constr) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
[#1] INFO:NumericIntegration -- RooRealIntegral::init(SR_model_Int[obs_x_SR]) using numeric integrator RooBinIntegrator to calculate Int(obs_x_SR)
[#1] INFO:Minization --  The following expressions have been identified as constant and will be precalculated and cached: (S_SR_nominal,B_SR_overallSyst_x_Exp)
Minuit2Minimizer: Minimize with max-calls 1000 convergence for edm < 1 strategy 0
[#1] INFO:Minization -- RooNLLVar::evaluatePartition(SR) first = 0 last = 1 Likelihood offset now set to -33.6584
Minuit2Minimizer : Valid minimum - status = 0
FVAL  = -3.41267603487242166
Edm   = 6.57632705355509438e-06
Nfcn  = 22
Lumi	  = 1.00002	 +/-  0.0191933	(limited)
mu	  = 1.59399	 +/-  0.776528	(limited)


Info in <Minuit2>: MnSeedGenerator Initial state: FCN =      -3.044377767 Edm =       0.424292289 NCalls =      9
Info in <Minuit2>: VariableMetricBuilder Start iterating until Edm is < 0.001 with call limit = 1000
Info in <Minuit2>: VariableMetricBuilder    0 - FCN =      -3.044377767 Edm =       0.424292289 NCalls =      9
Info in <Minuit2>: VariableMetricBuilder    1 - FCN =      -3.410907908 Edm =    0.001983649396 NCalls =     17
Info in <Minuit2>: VariableMetricBuilder    2 - FCN =      -3.412676035 Edm =   6.576327054e-06 NCalls =     22
Info in <RooFitResult::SaveAs>: ROOT file result.root has been created


# Excercises

1. Try runing the fit using the Asimov dataset ; what do you get for mu ?
2. Adjust the number of signal and background events (in step 1), re-run the fit and confirm that it behaves as you'd expect